In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime
import calendar
from scipy import interpolate
import os
import csv

In [2]:
def pcoord(x, y): 
    """
    Convert x, y to polar coordinates r, az (geographic convention)
    r,az = pcoord(x, y)
    """
    r  = np.sqrt( x**2 + y**2 )
    az=np.degrees( np.arctan2(x, y) )
    # az[where(az<0.)[0]] += 360.
    az = (az+360.)%360.
    return r, az

def xycoord(r, az):
    """
    Convert r, az [degrees, geographic convention] to rectangular coordinates
    x,y = xycoord(r, az)
    """
    x = r * np.sin(np.radians(az))
    y = r * np.cos(np.radians(az))
    return x, y


In [3]:
fn = 'buoydata/micity2017.txt'

df = pd.read_csv(fn, skiprows=range(1,2), delim_whitespace = True, \
                    parse_dates={'date':[0,1,2,3,4]}, keep_date_col=False)

# Transfer data in "date" column to a column where it is stored as a datetime object
df['datetime'] = pd.to_datetime(df['date'], format = '%Y %m %d %H %M',utc=True)
df = df.drop(df.columns[[0,1,2,3,6,8,9,10,11,12, 13]], axis = 1)
df

,WVHT,DPD,MWD,datetime
0,0.01,11.64,999,2017-04-19 18:10:00+00:00
1,0.01,99.00,999,2017-04-19 18:20:00+00:00
2,0.01,12.15,45,2017-04-19 18:30:00+00:00
3,0.01,10.41,40,2017-04-19 18:40:00+00:00
4,0.01,6.58,227,2017-04-19 18:50:00+00:00
...,...,...,...,...
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00
27894,0.92,2.29,999,2017-11-11 17:10:00+00:00
27895,0.91,2.29,999,2017-11-11 17:20:00+00:00
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00


In [4]:
# calculate unix datetime
df['epoch']=(df['datetime'] - pd.Timestamp("1970-01-01",tz='utc')) // pd.Timedelta('1s')
df

,WVHT,DPD,MWD,datetime,epoch
0,0.01,11.64,999,2017-04-19 18:10:00+00:00,1492625400
1,0.01,99.00,999,2017-04-19 18:20:00+00:00,1492626000
2,0.01,12.15,45,2017-04-19 18:30:00+00:00,1492626600
3,0.01,10.41,40,2017-04-19 18:40:00+00:00,1492627200
4,0.01,6.58,227,2017-04-19 18:50:00+00:00,1492627800
...,...,...,...,...,...
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00,1510419600
27894,0.92,2.29,999,2017-11-11 17:10:00+00:00,1510420200
27895,0.91,2.29,999,2017-11-11 17:20:00+00:00,1510420800
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00,1510421400


In [5]:
# remove data with NaN values
df = df[df['WVHT'] < 99.0]
df = df[df['DPD'] <99.0]
df = df[df['MWD'] < 999]
df

,WVHT,DPD,MWD,datetime,epoch
2,0.01,12.15,45,2017-04-19 18:30:00+00:00,1492626600
3,0.01,10.41,40,2017-04-19 18:40:00+00:00,1492627200
4,0.01,6.58,227,2017-04-19 18:50:00+00:00,1492627800
5,0.01,7.47,36,2017-04-19 19:00:00+00:00,1492628400
6,0.01,7.63,269,2017-04-19 19:10:00+00:00,1492629000
...,...,...,...,...,...
27888,0.43,2.22,202,2017-11-11 16:10:00+00:00,1510416600
27891,0.48,2.29,190,2017-11-11 16:40:00+00:00,1510418400
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00,1510419600
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00,1510421400


In [6]:
# get only data from October 2017
octdf = df[pd.DatetimeIndex(df['datetime']).month == 10]
octdf

,WVHT,DPD,MWD,datetime,epoch
21970,0.74,4.00,2,2017-10-01 00:50:00+00:00,1506819000
21972,0.68,3.46,31,2017-10-01 01:10:00+00:00,1506820200
21974,0.74,3.33,42,2017-10-01 01:30:00+00:00,1506821400
21980,0.64,3.51,288,2017-10-01 02:30:00+00:00,1506825000
21982,0.57,3.67,38,2017-10-01 02:50:00+00:00,1506826200
...,...,...,...,...,...
26352,0.60,5.47,320,2017-10-31 22:40:00+00:00,1509489600
26353,0.58,5.20,243,2017-10-31 22:50:00+00:00,1509490200
26355,0.60,5.01,77,2017-10-31 23:10:00+00:00,1509491400
26357,0.60,5.23,298,2017-10-31 23:30:00+00:00,1509492600


In [7]:
# put data frame into a numpy array
waves = octdf[octdf.columns[[4,0,1,2]]].to_numpy()
print('array with shape',np.shape(waves))
print(waves[0])

array with shape (2787, 4)
[1.506819e+09 7.400000e-01 4.000000e+00 2.000000e+00]


In [13]:
waves[:,0]

array([1.5068190e+09, 1.5068202e+09, 1.5068214e+09, ..., 1.5094914e+09,
       1.5094926e+09, 1.5094938e+09])

In [10]:
#check first time in record
(pd.Timestamp("2017-10-01 00:50:00")-pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

1506819000

In [14]:
# interpolators

#linear interpolator
linf = interpolate.interp1d(waves[:,0], waves[:,1:4], axis=0, kind='linear')
# Nearest Neighbor interpolator
nnf =interpolate.interp1d(waves[:,0], waves[:,0:4], axis=0, kind='nearest')

In [48]:
# write the csv file 

fields = ["imagename", "waveheight", "waveper", "wavedir"]

#  naming convention is Michigan City, View 2, October 2017
with open("MCv2oct2017.csv", "w") as fn:
    csvw = csv.writer(fn, delimiter = ",")
    csvw.writerow(fields)

AttributeError: '_io.TextIOWrapper' object has no attribute 'writer'

In [39]:
# convert image file name to Unix epox time
for filename in os.listdir("D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017"):
    size = len(filename)
    # Slice string to remove last 3 characters from string
    filename = filename[:size - 4]
    ti = calendar.timegm(datetime.datetime.strptime(filename, "%Y%m%d%H%M").timetuple())
    print(ti)
    wi = linf(ti)
    print(wi)

1507149180
[  0.884   4.341 114.6  ]
1509197580
[  1.384   4.258 299.   ]
1509042780
[  0.481   2.391 191.   ]
1507069980
[  0.505   2.717 256.1  ]
1509039180
[  0.427   2.309 180.4  ]
1508941980
[  1.742   7.726 245.4  ]
1509301980
[0.911 5.944 9.8  ]
1508168040
[  1.29    5.314 118.8  ]
1507818840
[  0.6     4.976 220.4  ]
1508427240
[  0.506   3.374 293.4  ]
1507340040
[  0.31525    2.019125 195.7625  ]
1507426440
[  1.213   4.474 278.2  ]
1508290440
[  0.41466667   3.23466667 271.        ]
1508204040
[  0.448   5.426 202.6  ]
1507854840
[ 0.22   4.772 19.8  ]
1509463980
[  1.5755   5.507  216.7   ]
1508013180
[ 0.205875  2.02     59.725   ]
1506882840
[  0.35004878   2.         132.11219512]
1507232040
[  0.398   4.358 170.2  ]
1507563240
[1.37125000e-01 2.02493750e+00 2.01065625e+02]
1508790840
[  1.79   5.58 213.6 ]
1506879240
[  0.35297561   2.         138.84390244]
1506954840
[  0.366   2.136 200.2  ]
1507077180
[  0.528   2.492 249.2  ]
1508949240
[  1.462   7.744 143.6  ]
150

ValueError: A value in x_new is below the interpolation range.

In [46]:
for filename in os.listdir("D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017"):
    size = len(filename)
    # Slice string to remove last 3 characters from string
    filename = filename[:size - 4]
    ti = calendar.timegm(datetime.datetime.strptime(filename, "%Y%m%d%H%M").timetuple())
    print(ti)        
    # note that values from all columns are interpolated.
    zi = nnf(ti)
    print(zi)
    timedif = abs(int(ti)-int(zi[0]))
    print (timedif)
    cieling = 30*60
    flag = timedif<=cieling
    print(flag)

1507149180
[1.507149e+09 8.900000e-01 4.320000e+00 2.700000e+01]
180
True
1509197580
[1.5091974e+09 1.3900000e+00 4.2400000e+00 3.0200000e+02]
180
True
1509042780
[1.5090426e+09 4.8000000e-01 2.4000000e+00 1.9200000e+02]
180
True
1507069980
[1.5070698e+09 5.1000000e-01 2.6900000e+00 2.5600000e+02]
180
True
1509039180
[1.509039e+09 4.300000e-01 2.300000e+00 1.840000e+02]
180
True
1508941980
[1.5089418e+09 1.7600000e+00 7.7500000e+00 3.4800000e+02]
180
True
1509301980
[1.5093018e+09 9.1000000e-01 5.9100000e+00 1.0000000e+01]
180
True
1508168040
[1.5081678e+09 1.3300000e+00 5.2900000e+00 4.0000000e+00]
240
True
1507818840
[1.5078186e+09 6.0000000e-01 5.0400000e+00 3.5800000e+02]
240
True
1508427240
[1.5084276e+09 5.0000000e-01 3.3700000e+00 2.9800000e+02]
360
True
1507340040
[1.5073392e+09 3.1000000e-01 2.0200000e+00 1.9900000e+02]
840
True
1507426440
[1.5074268e+09 1.2100000e+00 4.3900000e+00 2.8900000e+02]
360
True
1508290440
[1.5082902e+09 4.2000000e-01 3.2600000e+00 2.7100000e+02]
240

ValueError: A value in x_new is below the interpolation range.